In [1]:
import os
import time
import random
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
service = Service("geckodriver.exe")
opt = webdriver.FirefoxOptions()
opt.binary_location = "C:\\Program Files\\Mozilla Firefox\\firefox.exe"
driver = webdriver.Firefox(service=service, options=opt)

In [3]:
page = 1
film_id = 258687
review_url = "https://www.kinopoisk.ru/film/{}/reviews/ord/rating/status/all/perpage/200/page/{}/"
target_url = review_url.format(film_id, page)
driver.get(target_url)
html = driver.page_source

In [3]:
def get_review_type(classes):
    if "good" in classes:
        return "POSITIVE"
    if "bad" in classes:
        return "NEGATIVE"
    return "NEUTRAL"

In [4]:
RU_MONTH_VALUES = {
    'января': 1,
    'февраля': 2,
    'марта': 3,
    'апреля': 4,
    'мая': 5,
    'июня': 6,
    'июля': 7,
    'августа': 8,
    'сентября': 9,
    'октября': 10,
    'ноября': 11,
    'декабря': 12,
}

def int_value_from_ru_month(date_str):
    for k, v in RU_MONTH_VALUES.items():
        date_str = date_str.replace(k, str(v))
    return date_str

def kinopoisk_date_str_to_datetime(date_str):
    date_str = int_value_from_ru_month(date_str)
    return datetime.strptime(date_str, "%d %m %Y | %H:%M")

In [5]:
def get_review_info(review):
    review_id = review["data-id"]
    review_type_data = review.findChild("div", {"itemprop": "reviews"})
    review_type = get_review_type(review_type_data["class"])
    review_text_data = review.findChild("div", {"class": "brand_words"})
    review_text = review_text_data.text
    review_title = review.findChild("p", {"class": "sub_title"}).text
    pos_and_neg_data = review.findChild("li", {"id": f"comment_num_vote_{review_id}"})
    pos, neg = map(int, pos_and_neg_data.text.replace("/", "").split())
    author_data = review.findChild("p", {"class": "profile_name"})
    author_id_data = author_data.findChild("a")["href"]
    author_id = int(author_id_data.lstrip("/user/").rstrip("/"))
    author_name = author_data.text
    date_raw = review.findChild("span", {"class": "date"}).text
    date = kinopoisk_date_str_to_datetime(date_raw)
    return (
        review_id, author_id, author_name, review_title, 
        review_type, pos, neg, review_text, date
    )

In [ ]:
soup = BeautifulSoup(html)
reviews = soup.find_all("div", {"class": ["reviewItem", "userReview"]})
len(reviews)

In [24]:
r_id, a_id, a_name, r_title, r_type, pos, neg, r_text, date = get_review_info(reviews[0])
r_id, a_id, a_name, r_title, r_type, pos, neg

('2131878', 1414072, 'Lost__Soul', 'Теория всего', 'POSITIVE', 882, 246)

In [6]:
def parse_reviews(soup: BeautifulSoup):
    film_title_raw = soup.findChild("a", {"class", "breadcrumbs__link"})
    film_id = int(film_title_raw["href"].lstrip("/film").rstrip("/"))
    film_title = film_title_raw.text
    reviews = soup.find_all("div", {"class": ["reviewItem", "userReview"]})
    reviews_info = [(film_id, film_title) + get_review_info(r) for r in reviews]
    columns = [
        "film_id", "film_title", "review_id", "author_id", "author_name", 
        "review_title", "review_type", "pos", "neg", "review_text", "date"
    ]
    return pd.DataFrame(data=reviews_info, columns=columns)

In [28]:
reviews = parse_reviews(soup)
reviews

,film_id,film_title,review_id,author_id,author_name,review_title,review_type,pos,neg,review_text,date
0,258687,Интерстеллар,2131878,1414072,Lost__Soul,Теория всего,POSITIVE,882,246,"\nКак же трудно, наверное, не сломаться под не...",2014-10-29 22:10:00
1,258687,Интерстеллар,2175036,5178279,Duke_Cheb,Ненаучная научная фантастика,NEGATIVE,506,243,"\nНе могу и я свои пять копеек не вставить, хо...",2015-01-19 13:29:00
2,258687,Интерстеллар,2135366,1904923,TheGreatCritic,"Событие, которое нельзя пропустить",POSITIVE,346,120,\n 'Интерстеллар' – это фантастический фильм...,2014-11-06 18:19:00
3,258687,Интерстеллар,2132150,1623867,DikCinema,История любви на фоне грандиозного космическог...,POSITIVE,344,133,"\nЛюбовь - единственное, что способно противос...",2014-10-30 18:04:00
4,258687,Интерстеллар,2131964,1876843,Кирилл Васин,Космос нас ждет,POSITIVE,308,116,"\nДоброго времени суток, дамы и господа. \n\nС...",2014-10-30 01:55:00
...,...,...,...,...,...,...,...,...,...,...,...
195,258687,Интерстеллар,2182560,2158817,mike_onkyo,Глобальная фантастика!,POSITIVE,13,4,\nСложно оценивать такие патетичные фильмы. Он...,2015-02-03 02:53:00
196,258687,Интерстеллар,2139724,100799,Марина Жукова,,POSITIVE,13,4,"\nИ первый мы сделаем, переступив порог киноте...",2014-11-13 10:39:00
197,258687,Интерстеллар,2136571,1422579,Stanislav Malkov,"Все, что может произойти - произойдет",POSITIVE,28,17,\nЭта история происходит в недатированном буду...,2014-11-08 10:58:00
198,258687,Интерстеллар,2229624,5608335,Drake 195,Новый уровень,POSITIVE,19,9,"\nОчень много слышал о фильме, прежде чем его ...",2015-05-03 00:30:00


In [7]:
def get_film_reviews_at_page(driver, film_id, page):
    review_url = "https://www.kinopoisk.ru/film/{}/reviews/ord/rating/status/all/perpage/200/page/{}/"
    driver.get(review_url.format(film_id, page))
    html = driver.page_source
    soup = BeautifulSoup(html)
    return parse_reviews(soup)

In [8]:
def rand(min, max):
    return random.random() * (max - min) + min

In [9]:
class ScrapingDetected(Exception):
    pass

In [10]:
def get_film_reviews_in_selenium(driver, film_id, show_progress=False):
    review_url = "https://www.kinopoisk.ru/film/{}/reviews/ord/rating/status/all/perpage/200/page/{}/"
    driver.get(review_url.format(film_id, 1))
    html = driver.page_source
    soup = BeautifulSoup(html)
    try:
        soup.findChild("a", {"class", "breadcrumbs__link"})
    except Exception:
        raise ScrapingDetected()
    try:
        review_count_raw = soup.find("li", {"class": "all"})
        review_count = int(review_count_raw.findChild("b").text)
        pages_count = int(np.ceil(review_count / 200))
    except Exception as e:
        #print(f"\r\n{e}")
        return
    total_reviews = [parse_reviews(soup)]
    if show_progress:
        print(f"\r1/{pages_count}", end="")
    for page in range(2, pages_count + 1):
        if show_progress:
            print(f"\r{page}/{pages_count}", end="")
        try:
            reviews_at_page = get_film_reviews_at_page(driver, film_id, page)
        except Exception as e:
            #print(f"\r\n{e}")
            break
        total_reviews.append(reviews_at_page)
    if show_progress:
        print()
    return pd.concat(total_reviews, ignore_index=True)

In [11]:
def save_chunk(reviews, film_ids, index, save_interval, temp_dir="temp"):
    sidx, eidx = index - save_interval, index
    chunk_fids = film_ids[sidx:eidx]
    chunk = reviews[reviews["film_id"].isin(chunk_fids)]
    chunk.to_csv(f"{temp_dir}/chunk_{sidx}_{eidx}.csv", index=False)

In [12]:
def get_films_reviews_in_selenium(driver, film_ids, show_progress=False, temp_dir="temp", chunk_size=0.1):
    result = []
    save_interval = int(np.floor(len(film_ids) * chunk_size))
    if not os.path.exists("temp"):
        os.mkdir("temp")
    try:
        for i, film_id in enumerate(film_ids, start=1):
            if i % save_interval == 0:
                total_reviews = pd.concat(result, ignore_index=True)
                save_chunk(total_reviews, film_ids, i, save_interval, temp_dir=temp_dir)
            if show_progress:
                print(f"\r{i}/{len(film_ids)}", end="")
            try:
                reviews = get_film_reviews_in_selenium(driver, film_id)
            except ScrapingDetected:
                return pd.concat(result, ignore_index=True)
            except Exception as e:
                # print(f"\r\n{e}")
                break
            if reviews is None:
                continue
            result.append(reviews)
        if show_progress:
            print()
    except KeyboardInterrupt:
        if show_progress:
            print()
        print("Stopping")
        pass
    if show_progress:
        print()
    result = pd.concat(result, ignore_index=True)
    try:
        last_interval = len(film_ids) - int(save_interval * 1 / chunk_size)
        save_chunk(result, film_ids, len(film_ids), last_interval, temp_dir=temp_dir)
    except Exception as e:
        print(e)
    return result

In [59]:
df = pd.read_csv("kion_first_254_films_reviews.csv")
df

,film_id,film_title,review_id,author_id,author_name,review_title,review_type,pos,neg,review_text,date
0,477647,Гостья,1755958,1414072,Lost__Soul,Прекрасное создание,NEGATIVE,241,123,\nНебезызвестный феномен современного кинемато...,2013-03-28 00:04:00
1,477647,Гостья,1755392,17062,rain 13,Идеальный мир паразитов,POSITIVE,138,62,"\nЕсли живы воспоминания, жив и ты (с)\n\nТема...",2013-03-27 11:21:00
2,477647,Гостья,1757153,423716,Aeger_Faber,Странница,NEGATIVE,153,90,"\nЕсли вы читаете эту рецензию, то, вероятно, ...",2013-03-29 16:59:00
3,477647,Гостья,1756803,1749081,клементина кручински,Копеечное счастье Стефани Майер.,NEGATIVE,141,84,"\nВсем, кто купился на мерцающий электрический...",2013-03-28 23:54:00
4,477647,Гостья,1761571,1685,korsar45,Безаналитическая история Говарда Лавью,NEUTRAL,46,12,\nШёл 2017-й год. Земля стремительно очищалась...,2013-04-04 19:33:00
...,...,...,...,...,...,...,...,...,...,...,...
5017,1038154,Три богатыря и принцесса Египта,2633406,1287125,dmutrod,Вот так Новый год!,POSITIVE,70,97,"\nКаждый Новый год все мы ожидаем чуда, которо...",2017-12-28 12:00:00
5018,1038154,Три богатыря и принцесса Египта,2634042,5375969,DzinDzon,Куда только не ступала нога Богатыря русского…,POSITIVE,55,90,\nГде только не бывали Три богатыря за долгие ...,2017-12-30 12:31:00
5019,1038154,Три богатыря и принцесса Египта,2638604,14555712,\nдумающий\n,Традиция. А что плохого?,POSITIVE,4,36,"\nНовый мультфильм про богатырей. Вновь, как в...",2018-01-12 16:25:00
5020,1038154,Три богатыря и принцесса Египта,2634702,5293971,Koropusina,Дурило!.. Не дури!,POSITIVE,28,82,\nНовые приключения полюбившихся нашим зрителя...,2018-01-02 16:48:00


In [24]:
fid = all_kion_films_except_top250_in_kinopoisk_films_ids
i = 163
save_interval = 163
sidx, eidx = i - save_interval, i
sidx, eidx

(0, 163)

In [34]:
chunk = df[df["film_id"].isin(fid[sidx:eidx])]
chunk

,film_id,film_title,review_id,author_id,author_name,review_title,review_type,pos,neg,review_text,date
0,477647,Гостья,1755958,1414072,Lost__Soul,Прекрасное создание,NEGATIVE,241,123,\nНебезызвестный феномен современного кинемато...,2013-03-28 00:04:00
1,477647,Гостья,1755392,17062,rain 13,Идеальный мир паразитов,POSITIVE,138,62,"\nЕсли живы воспоминания, жив и ты (с)\n\nТема...",2013-03-27 11:21:00
2,477647,Гостья,1757153,423716,Aeger_Faber,Странница,NEGATIVE,153,90,"\nЕсли вы читаете эту рецензию, то, вероятно, ...",2013-03-29 16:59:00
3,477647,Гостья,1756803,1749081,клементина кручински,Копеечное счастье Стефани Майер.,NEGATIVE,141,84,"\nВсем, кто купился на мерцающий электрический...",2013-03-28 23:54:00
4,477647,Гостья,1761571,1685,korsar45,Безаналитическая история Говарда Лавью,NEUTRAL,46,12,\nШёл 2017-й год. Земля стремительно очищалась...,2013-04-04 19:33:00
...,...,...,...,...,...,...,...,...,...,...,...
3254,987,Поймать вора,1369829,23617,V.A.N.,NaN,POSITIVE,14,10,\nФильм мне понравился. Отличный детективный ...,2011-11-16 09:04:00
3255,987,Поймать вора,3074186,75502912,Мария Миронова - 4209,NaN,POSITIVE,2,0,\nЗамечательное старое кино на вечер. Начала с...,2021-10-01 11:30:00
3256,987,Поймать вора,3043311,79502386,jroowellly@gmail.com,"Классика, котороя не стоит твоего времени",NEUTRAL,4,3,\nНе хочу начинать свою рецензию достаточно тр...,2021-07-01 04:45:00
3257,987,Поймать вора,3183642,426194,ars-projdakov,NaN,POSITIVE,0,0,\nАльфред Хичкок по праву считается одним из в...,2022-09-29 23:56:00


In [37]:
len(chunk["film_id"].unique()), len(fid[sidx:eidx])

(117, 163)

In [ ]:
save_chunk(result, film_ids, len(film_ids), last_interval, temp_dir=temp_dir)

In [13]:
top250 = pd.read_csv("top250.csv")
top250_ids = top250["FilmId"].values

In [ ]:
top250reviews = get_films_reviews_in_selenium(driver, top250_ids, show_progress=True)
top250reviews

In [27]:
top250reviews.to_csv("top250_films_reviews_ALL.csv", index=False)

In [14]:
kinopoisk = pd.read_csv("kinopoisk.csv")

c:\Program Files\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (1,6,10,11,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
kinopoisk.head()

,Unnamed: 0,kinopoisk_id,age_limit,types,title_ru,title_orig,release_year,countries,genres,director,...,mounted_in,mounted_from,parodied_in,spin_off,spin_off_from,is_mentioned_in,agg_content_gid,similar_agg_content_gid,gr_content_id,business_dt
0,0.0,273588,NaN,film,Обручальное кольцо,L'anello matrimoniale,1979.0,"['Италия', 'Испания']",['комедия'],['Мауро Ивальди'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-29
1,1.0,91319,NaN,film,Большие универмаги,I grandi magazzini,1939.0,['Италия'],['драма'],['Марио Камерини'],...,NaN,NaN,NaN,NaN,NaN,['https://www.kinopoisk.ru/film/25721/'],NaN,NaN,NaN,2022-11-29
2,2.0,1429440,NaN,series,Giochi Sporchi,Giochi Sporchi,2009.0,['Италия'],['криминал'],['David Emmer'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-29
3,3.0,1094303,NaN,film,Quaternion,Quaternion,1976.0,['США'],['короткометражка'],['Холлис Фрэмптон'],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-29
4,4.0,1412420,NaN,film,Dead or Alive: That&apos;s the Way,Dead or Alive: That's the Way (I Like It),1984.0,['Великобритания'],['короткометражка'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-29


In [16]:
kinopoisk["types"].unique()

array(['film', 'series', nan, '4200000 $', '2500000 €', '350000 $',
       '20000000 $', '2000000 $', '1000000 $', '10000000 $', '3000000 $',
       '6000000 $', '92000000 $', '20000 $', '4000000 $', '3000000 CA$',
       '55000000 $', '350000 €', '22000000 $'], dtype=object)

In [17]:
kion_films = kinopoisk[(~kinopoisk["gr_content_id"].isna()) & (kinopoisk["types"] == "film")]
all_kion_films_except_top250_in_kinopoisk = kion_films[~kion_films["kinopoisk_id"].isin(top250_ids)]
all_kion_films_except_top250_in_kinopoisk_films_ids = all_kion_films_except_top250_in_kinopoisk["kinopoisk_id"].values.astype(int)
all_kion_films_except_top250_in_kinopoisk_films_ids

array([  43141,  477647, 1158168, ..., 1445067,  397298,  775410])

In [18]:
all_kion_films_reviews_except_top250_in_kinopoisk = get_films_reviews_in_selenium(driver, all_kion_films_except_top250_in_kinopoisk_films_ids, show_progress=True, chunk_size=0.05)
all_kion_films_reviews_except_top250_in_kinopoisk

16306/16306



,film_id,film_title,review_id,author_id,author_name,review_title,review_type,pos,neg,review_text,date
0,477647,Гостья,1755958,1414072,Lost__Soul,Прекрасное создание,NEGATIVE,241,123,\nНебезызвестный феномен современного кинемато...,2013-03-28 00:04:00
1,477647,Гостья,1755392,17062,rain 13,Идеальный мир паразитов,POSITIVE,138,62,"\nЕсли живы воспоминания, жив и ты (с)\n\nТема...",2013-03-27 11:21:00
2,477647,Гостья,1757153,423716,Aeger_Faber,Странница,NEGATIVE,153,90,"\nЕсли вы читаете эту рецензию, то, вероятно, ...",2013-03-29 16:59:00
3,477647,Гостья,1756803,1749081,клементина кручински,Копеечное счастье Стефани Майер.,NEGATIVE,141,84,"\nВсем, кто купился на мерцающий электрический...",2013-03-28 23:54:00
4,477647,Гостья,1761571,1685,korsar45,Безаналитическая история Говарда Лавью,NEUTRAL,46,12,\nШёл 2017-й год. Земля стремительно очищалась...,2013-04-04 19:33:00
...,...,...,...,...,...,...,...,...,...,...,...
397803,775410,Виселица,2622075,6943250,\nLionheart92\n,,NEUTRAL,19,5,\nВремя от времени Аль Пачино снимается в схож...,2017-11-26 16:55:00
397804,775410,Виселица,2757625,2026005,cyberlaw,Виселица с Аль Пачино,NEGATIVE,15,9,"\nВеликий актер. Он добился таких высот, что т...",2019-01-09 04:48:00
397805,775410,Виселица,2953490,2321465,Fozzy,Расследование убийств,POSITIVE,5,2,\nАль Пачино - легенда кинематографа (правда п...,2020-10-05 19:26:00
397806,775410,Виселица,2629158,1102751,neo1570,Затертая до дыр игра в маньяка.,NEGATIVE,25,23,"\nНовинка от режиссера Джонни Мартина, который...",2017-12-15 16:56:00


In [19]:
all_kion_films_reviews_except_top250_in_kinopoisk.to_csv("kion_films_reviews.csv", index=False)

In [20]:
len(all_kion_films_reviews_except_top250_in_kinopoisk["film_id"].unique())

11686

In [71]:
len(all_kion_films_reviews_except_top250_in_kinopoisk["film_id"].unique())

10